In [1]:
"""
This is the function you'll use to add an arbitrary cc table to your running cc history
"""

"\nThis is the function you'll use to add an arbitrary cc table to your running cc history\n"

In [2]:
import pandas as pd
import numpy as np

from openpyxl import load_workbook

import datetime

import glob

In [3]:
def get_dtime (string):
    """
    converts strings to datetime objects using datetime.strptime mostly just wrote this so I could .apply() it in pandas
    method chains
    
    takes (string) which is the full date of format mm/dd/yyyy (can't be 2 digit year)
    """
    return datetime.datetime.strptime(string, '%m/%d/%Y')

In [4]:
def get_chase_addendum (glob_list_of_cc_addenda):
    
    chase = pd.read_csv([z for z in glob_list_of_cc_addenda if 'chase' in z][0])
    
    chase_clean = (chase.drop(['Type', 'Post Date'], axis=1)
               .rename(columns = {'Trans Date' : 'date', 'Description' : 'desc', 'Amount' : 'price'})
               .assign(card = 'chase -8723')
              )
    
    chase_clean['price'] = chase_clean['price'].apply(np.negative)
    chase_clean['date'] = chase_clean['date'].apply(get_dtime)
    
    min_max_dates = [min(chase_clean['date']), max(chase_clean['date'])]
    
    
    return chase_clean, min_max_dates

In [5]:
filepath = '../../../Finances/cc info'

In [6]:
hist_files = glob.glob(filepath + '/*.csv')

In [7]:
#full cc hist to update
full_cc_hist = pd.read_csv([z for z in hist_files if 'tidy' in z][0])
full_cc_hist['date'] = full_cc_hist['date'].apply(get_dtime)

In [8]:
#get the properly formatted chase transactions to be added and the date range they span
chase_clean, min_max_date_range = get_chase_addendum(hist_files)

In [19]:
def find_new_entries (full_cc_hist, clean_new_entries, date_range, card_name):
    """
    Takes a properly column titled and formatted df of possible new entries
    and checks against existing transaction entries (within the date range spanned by new ones)
    for duplicates, dropping them all and returning only verified new entries to append to cc hist
    """
    
    #select all the transactions already in the history for this card between the dates the addenda span
    card_idx = full_cc_hist['card'] == card_name

    earlier_idx = full_cc_hist['date'] <= date_range[1]
    later_idx = full_cc_hist['date'] >= date_range[0]
    
    #get the existing transactions in this date range
    existing = full_cc_hist.loc[card_idx & earlier_idx & later_idx]
    
    #make a list of possible duplicate entries to check over
    possible_duplis = pd.concat([existing, chase_clean], sort=False)
    
    #get idx of ALL duplicates in the concatendated df
    dupli_idx = possible_duplis.duplicated(subset=['date', 'price', 'card'], keep=False)
    
    #create a df of just the duplicate entries (all of them)
    duplicate_entries = possible_duplis.loc[dupli_idx]
    
    #drop all the duplicates from the list (by index, since new ones have fresh index from 0, old ones have high index)
    new_entries = possible_duplis.drop(duplicate_entries.index)
    
    return new_entries

In [20]:
find_new_entries (full_cc_hist, chase_clean, min_max_date_range, 'chase -8723')

,date,price,desc,card,cat,st close,st open
0,2018-11-16,2.00,"PKG PS 1-6, KEC, 9",chase -8723,NaN,NaN,NaN
1,2018-11-06,1.25,CITY PASADENA,chase -8723,NaN,NaN,NaN
2,2018-11-05,-239.95,AUTOMATIC PAYMENT - THANK,chase -8723,NaN,NaN,NaN


I think i did this a dumb way, the simple way to do this is to find the latest transaction date for the card, then find all new transactions that happened on or after that date, only check for duplicates on that overlapping day, and add the non duplicates and later transactions. that's way better and much less error prone. duh